In [20]:
import cv2 as cv
import numpy as np

# ../../datasets/CuLane\test\driver_100_30frame\05250325_0272.MP4\00690.jpg
# ../../datasets/CuLane\test\driver_100_30frame\05250325_0272.MP4\00690.lines.txt

# EXAMPLE OF FALSE ROAD
# lines = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_37_30frame\05181520_0219.MP4\01200.lines.txt'
# image = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_37_30frame\05181520_0219.MP4\01200.jpg'

# lines = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_100_30frame\05250325_0272.MP4\00690.lines.txt'
# image = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_100_30frame\05250325_0272.MP4\00690.jpg'

lines = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_193_90frame\06060824_0802.MP4\01980.lines.txt'
image = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_193_90frame\06060824_0802.MP4\01980.jpg'

def data_roi_optimize_fixed_height(lines_file, image_shape, fixed_height=200, bottom_offset=70, padding=0):
    points = []

    with open(lines_file, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))
            for i in range(0, len(coords), 2):
                x = int(coords[i])
                y = int(coords[i + 1])
                points.append([x, y])

    img_height, img_width = image_shape[0], image_shape[1]

    total_height = fixed_height
    y_max = img_height - bottom_offset

    if y_max > img_height:
        y_max = img_height

    y_min = y_max - total_height

    if y_min < 0:
        y_min = 0
        y_max = y_min + total_height

    if padding > 0:
        y_min = max(y_min - padding, 0)
        y_max = min(y_max + padding, img_height)

    return 0, y_min, img_width, y_max


def data_process_mask(lines_file, image_file):
    mask_wdith, mask_height, shape = cv.imread(image_file).shape

    lines = []

    mask = np.zeros((mask_wdith, mask_height), dtype=np.uint8)

    line_id = lane_id = coords_first_id = coords_second_id = 0

    with open(lines_file, 'r') as file:
        lines = [list(map(int, map(float, line.split()))) for line in file]
    
    while line_id != len(lines) - 1:
        coords_first_id = coords_second_id = 0

        while coords_first_id + 4 <= len(lines[line_id]) or coords_second_id + 4 <= len(lines[line_id + 1]):
            x1, y1 = lines[line_id][coords_first_id], lines[line_id][coords_first_id + 1]
            x2, y2 = lines[line_id][coords_first_id + 2], lines[line_id][coords_first_id + 3]
            x3, y3 = lines[line_id + 1][coords_second_id], lines[line_id + 1][coords_second_id + 1]
            x4, y4 = lines[line_id + 1][coords_second_id + 2], lines[line_id + 1][coords_second_id + 3]

            cv.fillPoly(mask, [np.array([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])], lane_id + 1, lineType = cv.LINE_AA)

            if coords_first_id + 4 < len(lines[line_id]):
                    coords_first_id += 2
            
            if coords_second_id + 4 < len(lines[line_id + 1]):
                    coords_second_id += 2

            if coords_first_id + 4 == len(lines[line_id]) and coords_second_id + 4 == len(lines[line_id + 1]):
                    x1, y1 = lines[line_id][coords_first_id], lines[line_id][coords_first_id + 1]
                    x2, y2 = lines[line_id][coords_first_id + 2], lines[line_id][coords_first_id + 3]
                    x3, y3 = lines[line_id + 1][coords_second_id], lines[line_id + 1][coords_second_id + 1]
                    x4, y4 = lines[line_id + 1][coords_second_id + 2], lines[line_id + 1][coords_second_id + 3]

                    cv.fillPoly(mask, [np.array([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])], lane_id + 1, lineType = cv.LINE_AA)

                    break

        line_id += 1

        lane_id += 1

    return np.stack([mask] * 3, axis=-1), lane_id, (mask_wdith, mask_height, shape)

def resize_and_pad_mask(mask, target_size=312, pad_value=255):
    mask = mask[:, :, 0]

    original_height, original_width = mask.shape

    scale = min(target_size / original_height, target_size / original_width)
    new_height = int(round(original_height * scale))
    new_width = int(round(original_width * scale))

    resized = cv.resize(mask, (new_width, new_height), interpolation=cv.INTER_NEAREST)

    print(resized.shape)

    pad_top = (target_size - new_height) // 2
    pad_bottom = target_size - new_height - pad_top
    pad_left = (target_size - new_width) // 2
    pad_right = target_size - new_width - pad_left

    padded = cv.copyMakeBorder(
        resized,
        pad_top, pad_bottom,
        pad_left, pad_right,
        borderType=cv.BORDER_CONSTANT,
        value=pad_value
    )

    return padded.astype(np.uint8)

mask, lanes_num, img_shape = data_process_mask(lines, image)

x_min, y_min, x_max, y_max = data_roi_optimize_fixed_height(lines, img_shape, padding=0)

mask = mask[y_min:y_max, x_min:x_max]

# mask_res = resize_and_pad_mask(mask)

colored_mask = mask.copy()

print(mask.shape)

image_raw = cv.imread(image)

image_raw = image_raw[y_min:y_max, x_min:x_max]

print(f"x_min={x_min}, y_min={y_min}, x_max={x_max}, y_max={y_max}")
print(f"Crop height: {y_max - y_min}, Crop width: {x_max - x_min}")
print(f"Original image height: {img_shape[0]}")

# image_crop = image_raw[y_min:y_max, x_min:x_max]

def resize_and_pad_image(img, target_size=312, pad_value=127):
    original_height, original_width = img.shape[:2]
    scale = min(target_size / original_height, target_size / original_width)
    new_height = int(round(original_height * scale))
    new_width = int(round(original_width * scale))

    resized = cv.resize(img, (new_width, new_height), interpolation=cv.INTER_LINEAR)

    pad_top = (target_size - new_height) // 2
    pad_bottom = target_size - new_height - pad_top
    pad_left = (target_size - new_width) // 2
    pad_right = target_size - new_width - pad_left

    padded = cv.copyMakeBorder(
        resized,
        pad_top, pad_bottom,
        pad_left, pad_right,
        borderType=cv.BORDER_CONSTANT,
        value=(pad_value, pad_value, pad_value)
    )

    return padded

colored_mask = cv.applyColorMap(mask * (255 // (lanes_num + 2)), cv.COLORMAP_JET)

image_res = resize_and_pad_image(image_raw)

colored_mask_res = colored_mask.copy()

colored_mask_res = resize_and_pad_image(colored_mask_res)

cv.imshow("Image Original", image_raw)

cv.imshow("Mask Original", colored_mask)

cv.imshow("Image", image_res)

cv.imshow("Mask", colored_mask_res)

cv.waitKey(0)
cv.destroyAllWindows()

def data_absolute_roi_optimize(lines_file, image_shape, scale=1.2, padding=10):
    points = []

    with open(lines_file, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))
            for i in range(0, len(coords), 2):
                x = int(coords[i])
                y = int(coords[i + 1])
                points.append([x, y])

    if not points:
        return 0, 0, image_shape[1], image_shape[0]

    points_array = np.array(points)
    x, y, w, h = cv.boundingRect(points_array)

    cx = x + w // 2
    cy = y + h // 2

    new_w = int(w * scale)
    new_h = int(h * scale)

    new_x_min = max(cx - new_w // 2 - padding, 0)
    new_y_min = max(cy - new_h // 2 - padding, 0)
    new_x_max = min(cx + new_w // 2 + padding, image_shape[1])
    new_y_max = min(cy + new_h // 2 + padding, image_shape[0])

    return new_x_min, new_y_min, new_x_max, new_y_max

# import glob
# import os
# import numpy as np
# import cv2 as cv

# # lines = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_37_30frame\05181520_0219.MP4\00000.lines.txt'
# # image = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_37_30frame\05181520_0219.MP4\00000.jpg'

# def data_roi_optimize(lines_file, image_shape, scale=1.2, padding=10):
#     points = []

#     with open(lines_file, 'r') as file:
#         for line in file:
#             coords = list(map(float, line.strip().split()))
#             for i in range(0, len(coords), 2):
#                 x = int(coords[i])
#                 y = int(coords[i + 1])
#                 points.append([x, y])

#     if not points:
#         return 0, 0, image_shape[1], image_shape[0]

#     points_array = np.array(points)
#     x, y, w, h = cv.boundingRect(points_array)

#     cx = x + w // 2
#     cy = y + h // 2

#     new_w = int(w * scale)
#     new_h = int(h * scale)

#     new_x_min = max(cx - new_w // 2 - padding, 0)
#     new_y_min = max(cy - new_h // 2 - padding, 0)
#     new_x_max = min(cx + new_w // 2 + padding, image_shape[1])
#     new_y_max = min(cy + new_h // 2 + padding, image_shape[0])

#     return new_x_min, new_y_min, new_x_max, new_y_max

# def data_process_mask(lines_file, image_file):
#     mask_wdith, mask_height, shape = cv.imread(image_file).shape

#     lines = []

#     mask = np.zeros((mask_wdith, mask_height), dtype=np.uint8)

#     line_id = lane_id = coords_first_id = coords_second_id = 0

#     with open(lines_file, 'r') as file:
#         lines = [list(map(int, map(float, line.split()))) for line in file]
    
#     while line_id != len(lines) - 1:
#         coords_first_id = coords_second_id = 0

#         while coords_first_id + 4 <= len(lines[line_id]) or coords_second_id + 4 <= len(lines[line_id + 1]):
#             x1, y1 = lines[line_id][coords_first_id], lines[line_id][coords_first_id + 1]
#             x2, y2 = lines[line_id][coords_first_id + 2], lines[line_id][coords_first_id + 3]
#             x3, y3 = lines[line_id + 1][coords_second_id], lines[line_id + 1][coords_second_id + 1]
#             x4, y4 = lines[line_id + 1][coords_second_id + 2], lines[line_id + 1][coords_second_id + 3]

#             cv.fillPoly(mask, [np.array([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])], lane_id + 1, lineType = cv.LINE_AA)

#             if coords_first_id + 4 < len(lines[line_id]):
#                     coords_first_id += 2
            
#             if coords_second_id + 4 < len(lines[line_id + 1]):
#                     coords_second_id += 2

#             if coords_first_id + 4 == len(lines[line_id]) and coords_second_id + 4 == len(lines[line_id + 1]):
#                     x1, y1 = lines[line_id][coords_first_id], lines[line_id][coords_first_id + 1]
#                     x2, y2 = lines[line_id][coords_first_id + 2], lines[line_id][coords_first_id + 3]
#                     x3, y3 = lines[line_id + 1][coords_second_id], lines[line_id + 1][coords_second_id + 1]
#                     x4, y4 = lines[line_id + 1][coords_second_id + 2], lines[line_id + 1][coords_second_id + 3]

#                     cv.fillPoly(mask, [np.array([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])], lane_id + 1, lineType = cv.LINE_AA)

#                     break

#         line_id += 1

#         lane_id += 1

#     return np.stack([mask] * 3, axis=-1), lane_id, (mask_wdith, mask_height, shape)

# def resize_and_pad_mask(mask, target_size=312, pad_value=255):
#     mask = mask[:, :, 0]

#     original_height, original_width = mask.shape

#     scale = min(target_size / original_height, target_size / original_width)
#     new_height = int(round(original_height * scale))
#     new_width = int(round(original_width * scale))

#     resized = cv.resize(mask, (new_width, new_height), interpolation=cv.INTER_NEAREST)

#     pad_top = (target_size - new_height) // 2
#     pad_bottom = target_size - new_height - pad_top
#     pad_left = (target_size - new_width) // 2
#     pad_right = target_size - new_width - pad_left

#     padded = cv.copyMakeBorder(
#         resized,
#         pad_top, pad_bottom,
#         pad_left, pad_right,
#         borderType=cv.BORDER_CONSTANT,
#         value=pad_value
#     )

#     return padded.astype(np.uint8), resized.shape[0]

# lines = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_100_30frame\05250322_0271.MP4\00090.lines.txt'
# image = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_100_30frame\05250322_0271.MP4\00090.jpg'

# max_height_greatest = 0

# mask, lanes_num, img_shape = data_process_mask(lines, image)

# x_min, y_min, x_max, y_max = data_roi_optimize(lines, img_shape, scale=1, padding=0)

# mask = mask[y_min:y_max, x_min:x_max]

# mask_res, max_height = resize_and_pad_mask(mask)

# print(max_height)

# TARGET_DIR = r"../../datasets/CuLane"

# max_height_greatest = 0

# for filename in glob.glob(os.path.join(TARGET_DIR, "**", "*.jpg"), recursive=True):

#     lines_file = filename.replace(".jpg", ".lines.txt")

#     if not os.path.exists(lines_file):
#         continue

#     with open(lines_file, 'r') as f:
#         content = f.read().strip()
#     if not content:
#         continue

#     mask, lanes_num, img_shape = data_process_mask(lines_file, filename)

#     x_min, y_min, x_max, y_max = data_roi_optimize(lines_file, img_shape, scale=1, padding=0)

#     mask = mask[y_min:y_max, x_min:x_max]

#     mask_res, max_height = resize_and_pad_mask(mask)

#     if max_height_greatest < max_height:
#         max_height_greatest = max_height
    
#     print(filename)
#     print(lines_file)
#     print(max_height_greatest)

(200, 1640, 3)
x_min=0, y_min=320, x_max=1640, y_max=520
Crop height: 200, Crop width: 1640
Original image height: 590
